In [2]:
from telethon import TelegramClient
from datetime import timezone, timedelta, datetime
from api import *
import ktrain

/home/j/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
classifier = ktrain.load_predictor('../models/distilbert')

In [41]:
async def get_messages():
    global api_id
    global api_hash
    messages = []
    async with TelegramClient(api_id=api_id, api_hash=api_hash, session='this') as client:
        async for message in client.iter_messages('AmplifyUkraine', offset_date=datetime.now(tz=timezone.utc) - timedelta(hours=12), reverse=True):
            messages.append(message)
    return messages

def find_combats(messages):
    global classifier
    combats = []
    for message in messages:
        if classifier.predict(message.text) == 'annotation':
            combats.append(message)
    return combats

async def post_messages(messages):
    global api_id
    global api_hash
    async with TelegramClient(api_id=api_id, api_hash=api_hash, session='this') as client:
        for message in messages:
            await client.send_message(entity=-1002040681043, message=message.text)


In [44]:
async def main():
    messages = await get_messages()
    combats = find_combats(messages)
    await post_messages(combats)


In [46]:
await main()